# Vikhr 0.5_it beta, chatml 004, 1052

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers==4.36.2 -qq
!pip install accelerate -qq
!pip install SentencePiece -qq
!pip install bitsandbytes -qq
!pip install protobuf -qq
!pip install peft

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/SciKeywords_data/rukeywords_langdetect.csv")
new_texts = list(df.texts.values)
new_keywords = list(df.keywords.values)
train_texts = new_texts[:round(len(new_texts)*0.7)]
train_keywords = new_keywords[:round(len(new_texts)*0.7)]
test_texts = new_texts[round(len(new_texts)*0.7):]
test_keywords = new_keywords[round(len(new_texts)*0.7):]

In [ ]:
import pickle
with open('/content/gdrive/MyDrive/damdid 2024/llm/vikhr_math.pickle', 'rb') as f:
        generated_texts = pickle.load(f)
len(generated_texts)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4",
                                             device_map="auto",
                                             #attn_implementation="flash_attention_2",
                                             torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4")
from transformers import  AutoTokenizer, pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
prompt = "Сгенерируй ключевые слова для научной статьи по тексту аннотации. \n"

def add_examples(dsp, i):
  random.seed(i)
  for i in range(3):
    r = random.randint(0,len(train_texts)-1)
    dsp += 'Текст аннотации: '
    dsp += train_texts[r]
    dsp += ' \n'
    dsp += train_keywords[r]
    dsp += ' \n'
  return dsp

def test_inference(prompt, i):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": add_examples(prompt) + '\nТекст аннотации:\n' + test_texts[i]}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=100, do_sample=True, num_beams=1, temperature=0.25, top_k=50, top_p=0.98, eos_token_id=79097)
    return outputs[0]['generated_text'][len(prompt):].strip()


import pickle
#generated_texts =[]
for i in range(len(test_texts)):
  if i>2490:
    answer = test_inference(prompt, i)
    generated_texts.append(answer)
    if i%10==0 and i>0:
      print(i)
      print(test_texts[i])
      print(answer)
      with open('/content/gdrive/MyDrive/damdid 2024/llm/vikhr_math.pickle', 'wb') as f:
        pickle.dump(generated_texts, f)

In [ ]:
with open('/content/gdrive/MyDrive/damdid 2024/llm/vikhr_math.pickle', 'wb') as f:
        pickle.dump(generated_texts, f)

In [ ]:
len(generated_texts)

#FLAN

In [ ]:
!pip install accelerate
!pip install transformers

In [ ]:
import accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
#prompt = "Сгенерируй ключевые слова для научной статьи по тексту аннотации. Ключевые слова выведи в одну строку через запятую. \n Текст аннотации: "
prompt = 'Generate keywords for a scientific paper using the given abstract. Keyphrases are written in one line and separated by commas. \n Abstract:'
input_text = prompt + translation
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
keywords = tokenizer.decode(outputs[0])
print(keywords)

In [ ]:
input_text = "Translate Russian to English: " + test_texts[0]
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_new_tokens = 1024)
translation = tokenizer.decode(outputs[0])
print(translation)

In [ ]:
input_text = "Translate English to Russian: " + keywords
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_new_tokens = 128)
translated_keywords = tokenizer.decode(outputs[0])
print(translated_keywords)

In [ ]:
test_texts[0]